<a href="https://colab.research.google.com/github/svvsaga/datascience_workshop/blob/main/workshop_sesjon4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Workshop sesjon 4: Utforsking og visualisering



Hittil har vi sett på hvordan vi laster inn og tranformerer data i Google-plattformen og hvordan vi kan bruke BigQuery til å hente ut de dataene vi er interessert i og gjøre enkle analyser og aggregeringer. Nå skal vi se på et lett tilgjengelig verktøy for å utforske dataene videre og visualisere funnene våre - nemlig Data Studio.



## Utforsking i Data Studio

Vi tar utgangspunkt i den tidspartisjonerte tabellen vi opprettet i forrige økt. Denne gir oss en rad for hver kombinasjon av målepunktsidentifikator (```trpId```) og tidspunkt (```startOfHour``` og ```lokaltid```) med informasjon om antall kjøretøy som er målt (```volume```) og hvilken kommune målepunktet ligger i (```kommunenavn```).

Om du ikke har opprettet tabellen i slutten av sesjon 3, kan du kopiere og kjøre spørringen under. Husk å angi ditt eget prosjekt etter ```CREATE TABLE```!

I tillegg finnes en samsvarende tabell i vårt felles workshop-prosjekt (`saga-workshop-data-vu8x.workshop.produksjonstabell`) som du kan bruke i stedet.

```
CREATE TABLE `<workshop-project>.workshop.produksjonstabell` PARTITION BY DATE(startOfHour)
AS (
SELECT
  a.trpId,
  a.startOfHour,
  DATETIME(a.startOfHour,"Europe/Oslo") lokaltid,
  a.total.volumeNumbers.volume,
  c.kommunenavn,
  c.geometry
FROM
  `saga-workshop-data-vu8x.workshop.timestrafikkdata_v2` a
LEFT JOIN
  `saga-workshop-data-vu8x.workshop.trafikkregistreringspunkter` b
ON
  a.trpId = b.Id
JOIN
  `saga-workshop-data-vu8x.workshop.kommuner` c
ON
  ST_WITHIN( ST_GEOGPOINT(b.location.coordinates.latLon.lon, b.location.coordinates.latLon.lat), c.geometry)
)
```

Vi skal nå utforske disse dataene og se om vi kan finne noen påfallende mønstre, utliggere eller andre ting vi kan undersøke nærmere. Til det bruker vi verktøyet Data Studio, som er tett integrert med BigQuerys grafiske grensesnitt:


1.   Velg den tidspartisjonerte tabellen i Explorer-menyen til venstre i BigQuery-grensesnittet
2.   Klikk "Export" oppe til høyre
3.   Velg "Explore with Data Studio" i nedtrekksmenyen

Du skal nå få opp en ny fane i nettleseren med et nytt Data Studio-dokument. Foreløpig inneholder det bare en tabellvisning, men vi kan velge en rekke ulike diagrammer øverst i menyen til høyre. Her mistenker vi å kunne finne noe interessant ved å se på variasjon over tid, så vi velger *diagram over tidsrekker*.

Under diagramvelgeren har vi to faner med innstillinger – *Data* for å velge hvilke data vi vil vise og *Stil* for å formatere farger, akser og den slags. I *Data*-fanen er dette det viktigste vi må forholde oss til:


*   **Dimensjon**. Her angir vi hvilket felt vi vil knytte til x-aksen og med hvilken oppløsning. ```startOfHour```-feltet er allerede valgt for oss, så dette behøver vi ikke å endre på.
*   **Fordelingsdimensjon**. Her kan vi angi et felt som inneholder ulike kategorier dersom vi ønsker å vise hver kategori som en egen serie i diagrammet. Her har Data Studio valgt å vise hvert målepunkt som en egen serie, men det ønsker vi jo ikke. Vi fjerner derfor hele inndelingen i kategorier ved å sveve over feltnavnet og trykke på x-en som dukker opp. 
*   **Beregning**. Her knytter vi felter til y-aksen(e). Nå viser DataStudio bare ```Record Count```, altså antall rader tabellen har for hver dato. Vi klikker på feltnavnet og velger heller ```volume```. DataStudio skal nå velge å vise summen av ```volume```-feltene for hver dato, men vi kunne like gjerne valgt å vise gjennomsnitt, median, max eller noe annet ved å klikke på "SUM"-ikonet ved siden av feltnavnet.

<br>

Nå har vi en tidsserie som viser utviklingen i antall målte kjøretøy per døgn. Den skal se omtrent slik ut:

<img src='https://drive.google.com/uc?id=16dVMzeCBwM_b5oVCCwzBOBRvYW_BiEAH' width="900">

**Diskusjon**: Er det noen påfallende utviklingstrekk, mønstre eller andre ting vi kan legge merke til her?




### Diskusjonspunkter (spoiler alert!)

Det er en åpenbar **voksende trend** her - vi går jevnt og trutt fra null trafikk i 2014 til 20 millioner passeringer i 2019.
  * Hypotese: Det blir ikke mer trafikk, men flere *målinger*.
  * Vi undersøker dette ved å legge til en ny serie for antall målepunkter: Klikk og dra inn ```trpId```som nytt felt under *Beregning*. Gå så til *Stil*-fanen og legg serie 2 til høyre akse.
  * Hypotesen ser ut til å stemme - vi måler mer trafikk i takt med at det installeres flere tellepunkter.

Det er påfallende **regelmessighet i variasjonen** både på kort og mellomlang sikt.
  * Vi undersøker dette nærmere ved å se på kortere tidsperiode i større detalj.
  * Under *Standard dataperiode*, velg *Egendefinert*. Bruk så kalendergrensesnittet til å velge perioden fra 1. januar 2018 til 31. desember 2019.
  * Vi kan tydelig se effekten av ukedager, sommerferie, jul og påske.

<br>

**Konklusjon**: Det kan være interessant å undersøke disse systematiske variasjonene nærmere, men vi bør sammenligne de *samme* målepunktene over tid.


## Henting av et konsistent datasett

Vi ønsker å sammenligne data fra de *samme* målestasjonene over tid. Vi henter derfor en liste over alle målestasjonene og bruker ```GROUP BY```for å få én rad per målepunkt. Vi begrenser analysen til data fra 2018 til 2019 og angir dette med ```WHERE```.

Så bruker vi ```COUNT``` til å telle antall dager der vi har minst én måling fra et bestemt målepunkt. Dette kombinerer vi med ```HAVING```for å kun velge de målepunktene som har levert målinger i 730 dager, altså hver eneste dag i løpet av toårsperioden.

```
SELECT
    trpId
FROM
    `saga-workshop-data-vu8x.workshop.produksjonstabell`
WHERE
    lokaltid BETWEEN "2018-01-01" AND "2019-12-31" AND volume > 0
GROUP BY trpId
HAVING COUNT(DISTINCT DATE(lokaltid)) >= 730
```

Denne spørringen returnerer en liste over alle målestasjoner som leverte minst én måling hver dag i den perioden vi ser på. Vi vil nå ha med alle feltene i datasetet, men bare for disse målepunktene:

```
SELECT
    *
FROM `saga-workshop-data-vu8x.workshop.produksjonstabell`
WHERE lokaltid BETWEEN "2018-01-01" AND "2019-12-31" AND trpId IN (
SELECT
    trpId
FROM
    `saga-workshop-data-vu8x.workshop.produksjonstabell`
WHERE
    lokaltid BETWEEN "2018-01-01" AND "2019-12-31" AND volume > 0
GROUP BY trpId
HAVING COUNT(DISTINCT DATE(lokaltid)) >= 730
)
```

Vi kjører denne spørringen og utforsker resultatet i Data Studio - denne gangen ved å klike "Explore Data" fra rett over resultatet fra spørringen.

## Analyse av tidsvariasjon

Som sist har Data Studio valgt ```startOfHour``` som dimensjon. Denne angir tidspunkter i UTC, men vi skal gjøre analyser der tidssone og sommertid har en betydning, så vi endrer dette til ```lokaltid``` først som sist.

Under *Dimensjon* kan vi ikke bare velge hvilket felt vi vil bruke, men også tidsoppløsningen for aggregering. For å illustrere dette gjør vi følgende endringer i diagrammet vårt:
* Klikk på kalenderikonet ved siden av feltnavnet.
* Velg *Ukedag* fra menyen som dukker opp
* Gå til *Stil*-fanen, velg søylevisning for *Serie 1* og tving y-aksen til å begynne på 0

Vi har nå et diagram over totalt antall målte kjøretøy per ukedag gjennom de to årene vi ser på. Det skal se omtrent slik ut:

<img src='https://drive.google.com/uc?id=12cWJqdgurrsL17BVamXfWb_rBmyd6snG' width="800">

Enheten på y-aksen er nå i absolutte tall – men totalt antall målte kjøretøy for et vilkårlig sett med målepunkter er ikke særlig interessant å fortolke. Vi vil i stedet se hvilken *andel* av ukas totale trafikk som faller på hver ukedag. Det løser vi slik:
* Finn *Beregninger*-seksjonen i *Data*-fanen og klikk på SUM-ikonet ved siden av feltnavnet.
* Under *Sammenligningsberegning*, velg *Prosent av det totale antallet* og *I forhold til tilsvarende data*

Vi ser nå ukedagsvariasjonen for alle målepunkter i Norge under ett. For å kunne utforske forskjeller mellom kommunene ved å legge til et filter:
*   Finn ```kommunenavn``` i oversikten over tilgjengelige felt til høyre på skjermen
*   Dra feltet til *Filter*-boksen øverst på skjermen.
* Klikk på filteret, finn den kommunen du er interessert i og trykk på *Bare* og *Bruk*

Om vi ser på kun Oslo, skal vi nå ha noe slikt som dette:

<img src='https://drive.google.com/uc?id=1e5PvCG-Ib8WKbwq-ORx_sodALoF94Dl9' width="800">

<br>

Med disse instillingene på plass er det fort gjort å analysere andre typer tidsvariasjon. Ved å justere oppløsningen under *Dimensjon* kan vi for eksempel se på variasjon mellom måneder, ukenummer eller klokkeslett.

## Egendefinerte felter

Å bruke filter til å vise mer enn én kommune fungerer fint hvis vi vil se hvordan totalen for kommunene fordeler seg mellom ukedagene. Men om vi ønsker å vise kommunenes ukesfordelinger opp mot *hverandre*, må vi tenke litt annerledes. Her kan vi utnytte at Data Studio lar oss definere våre egne felter og lage et eget felt for hver kommune vi er interessert i:

* Slett filteret vi nettopp opprettet og velg bytt tilbake fra søyle- til linjevisning
* Under *Beregning*, trykk på ```volume``` og deretter *Opprett et felt*
* Gi feltet navnet Oslo
* Angi følgende formel: IF(kommunenavn="Oslo",volume,0)
* Under *Sammenligningsberegning*, velg *Prosent av det totale antallet*
* Klikk *Bruk* og legg til felter for Bergen, Trondheim, Kviteseid og Voss på samme måte

<br>

Nå skal nå ha en graf som gjør det lett å se likheter og forskjeller mellom ulike kommuner:

<img src='https://drive.google.com/uc?id=1Z3KNHZdZB5EQbaJKhqvWySt6WR2cenFA' width="800">

## BONUSDEL: Beregning av ukedagsfaktorer i BigQuery

Om vi ikke ønsker å lage egne felter for hver kommune vi ser på, kan vi løse dette på en annen måte. Det forutsetter at vi på forhånd vet a) hvilken tidsperiode vi ønsker å se på, og b) hvilken tidsoppløsning vi skal studere, altså om vi ønsker å aggregere per ukedag, kalendermåned, time på døgnet eller noe annet.

Det første vi gjør er å lagre et view basert på spørringen vi brukte til å trekke ut et konsistent datasett. Vi lagrer denne ved å trykke *Save* (ved siden av *Run*-knappen) og deretter *Save View*. Vi angir vårt personlige prosjekt og kaller viewet ```produksjonstabell_konsistent_2018-19```.

Nå har vi enkel tilgang til det konsistente datauttrekket og kan bruke dette til å beregne ukedagsandeler for hver kommune og ukedag. Vi lager først en spørring som aggregerer en rad for hver kombinasjon av kommunenavn og ukedag og summerer totalt trafikkvolum for denne kombinasjonen. Så spør vi på nytt etter totalt trafikkvolum for hvert kommunenavn *uavhengig* av ukedag. Til slutt kan vi joine disse spørringene og beregne forholdstallet mellom trafikkvolum per ukedag og trafikkvolum totalt. Vi får da et tall mellom 0 og 1 som angir hvor stor andel av ukas totale trafikk som faller på en gitt ukedag for en bestemt kommune.

```
WITH ukedagsaggregater AS (
SELECT
    kommunenavn,
    EXTRACT(DAYOFWEEK FROM lokaltid) as ukedag,
    SUM(volume) as ukedagstotal,
FROM `saga-workshop-data-vu8x.workshop.produksjonstabell_konsistent_2018-19` a
GROUP BY kommunenavn, ukedag
),
totalaggregater AS (
SELECT
    kommunenavn,
    SUM(volume) AS periodetotal
FROM `saga-workshop-data-vu8x.workshop.produksjonstabell_konsistent_2018-19` b
GROUP BY kommunenavn
)

SELECT
    kommunenavn,
    ukedag,
    ukedagstotal/periodetotal AS ukedagsandel
FROM ukedagsaggregater
LEFT JOIN totalaggregater
USING (kommunenavn)
```

Nå kan nå kjøre denne spørringne og åpne resultatet i Data Studio. Eksempelvis kan vi gjøre følgende for å vise en rangert oversikt over kommunene etter hvor stor andel av trafikken som foregår i helgene:


1.   Velg *Søylediagram* i diagramvelgeren
2.   Velg ```kommunenavn```som dimensjon
3. Velg ```ukedagsandel```under beregning
4. Legg til et filter for ```ukedag```-feltet - velg for eksempel å kun vise ukedag 1 (søndag)
5. Gå til Stil-fanen og velg antall kolonner (nesten øverst) - for eksempel 20

Vi har nå en oversikt over de kommunene i Norge (der vi har målinger) som har størst andel av ukestrafikken på søndager. Det bør se omtrent slik ut:

<img src='https://drive.google.com/uc?id=1uDZPVhruSWnq8ngFS4-Vzos_7mGFe-xc' width="800">



## Oppgave


1. Hent data for et konsistent utvalg av målepunkter
2.   Vis med en figur hvordan trafikken fordeler seg gjennom et typisk år for Norge totalt sett (for eksempel per måned eller ukenummer)
3.   Bruk et filter tiln å finne enkeltkommuner som skiller seg fra det nasjonale mønsteret
4. Sammenlign fordeling gjennom året for flere kommuner i én og samme figur (hint: egendefinerte felter)
5. Endre tidsoppløsningen og sammenlign de samme kommunene med tanke på døgn- og ukedags- eller andre variasjoner



## Løsningsforslag



1.   Se *Henting av konsistent datasett* over for én måte å gjøre det på.
2.   Her kan man velge et tidsseriediagram, velge ```volume``` under Beregning og sette tidsoppløsningen under Dimensjon til for eksempel Kvartal, Måned eller ISO-uke
3. Bruk ```kommunenavn```som filter. Her er det lurt å angi Sammenligningsberegning (Prosent av det totale antallet) under Beregning.
4. Se *Egendefinerte felter* over
5. Her trenger man kun å endre tidsoppløsningen i Dimensjon-feltet

Litt avhengig av hvordan man velger å løse det, kan resultatet for eksempel se slik ut:

<img src='https://drive.google.com/uc?id=1lBrUE4ViU0keM5Iw58VhDauoiYcuGx4R' width="800">

